In [ ]:
import sys
import logging
import concurrent.futures
import time
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pyproj import Transformer, CRS
from shapely.geometry import Polygon, Point
from sliderule import icesat2
from sliderule import sliderule
import geojson
from scipy.interpolate import griddata
from scipy.fft import fft, rfft, rfftfreq
from scipy.signal import blackman,detrend
import scipy.integrate as integrate
from pyproj import Transformer, Geod
import os
import pickle
import glob
import scipy
from scipy.stats import linregress
import io
import tqdm

In [ ]:
#open atl06 data
with open('/home/acdealy/notebooks/ice-shelf-roughness/sliderule_pig_big.pickle', 'rb') as handle:
    atl06_pig = pickle.load(handle)

In [ ]:
#define function that takes in height/distance vectors and outputs roughness with detrended psd
def get_roughness(h,d,k_lims):
    diffs = np.diff(d)
    sample_space = np.abs(np.mean(diffs))
    n = len(d)
    k = rfftfreq(n, sample_space)
    yf = sample_space * rfft(h)
    power = np.square(abs(yf))
    psd = power*(k[1]-k[0])
    psd = psd[(k>k_lims[0]) & (k<k_lims[1])]
    psd_detrend = detrend(psd)
    k = k[(k>k_lims[0]) & (k<k_lims[1])]
    if len(psd_detrend)>1:
        int_psd_detrend = np.max(integrate.cumulative_trapezoid(psd_detrend,k))
        R = np.sqrt(int_psd_detrend)
    else:
        R=np.nan
    return R,psd_detrend,k

#calculate roughness using sliding windows
def sliding_roughness(df,sz,slide,h_max=100,pts_per_km=10,pts_per_win=10,klims=[]):
    d, h, E, N = df["d"].values[0], df["h"].values[0], df["E"].values[0], df["N"].values[0]
    win_starts = np.arange(np.min(d),np.max(d),slide)
    R, R_detrend, E_out, N_out = [], [], [], []
    if klims == []:
        klims = [1/sz,1/(2*scipy.stats.mode(np.diff(d))[0][0])]    
    for i in tqdm.trange(len(win_starts)-1):
        win_idx = [(d > win_starts[i]) & (d < win_starts[i]+sz)][0]
        d_win, h_win, E_win, N_win = d[win_idx], h[win_idx], E[win_idx], N[win_idx]
        pts_per_km_data = (len(d_win)/sz)*1000
        pts_per_win_data = len(d_win)
        if ((pts_per_km_data>pts_per_km)&(pts_per_win_data>pts_per_win)):
            if np.sum(np.isnan(h_win))==0:
                h_detrend = detrend(h_win)
                roughness,_,_ = get_roughness(h_win,d_win,klims)
                roughness_detrend,_,_ = get_roughness(h_detrend,d_win,klims)
                R.append(roughness)
                R_detrend.append(roughness_detrend)
            else:
                R.append(np.nan)
                R_detrend.append(np.nan)
        else:
            R.append(np.nan)
            R_detrend.append(np.nan)
        E_out.append(np.nanmean(E_win))
        N_out.append(np.nanmean(N_win))
    return (R,R_detrend,E_out,N_out)

#concatenates individual track data into one large dataframe
def make_dataframe(atl06_data,cycle):
    atl06_data_cycle = atl06_data[atl06_data['cycle']==cycle]
    rgt_cycle_list = np.unique(atl06_data_cycle['rgt'])
    strong_spot_list = [1,3,5]
    df = pd.DataFrame(columns=["cycle","rgt","spot","d","h","E","N","R"],index=[])
    for rgt in rgt_cycle_list:
        for spot in strong_spot_list:
            h = atl06_data_cycle[(atl06_data_cycle['spot']==spot)&(atl06_data_cycle['rgt']==rgt)].h_mean.values
            lat = atl06_data_cycle[(atl06_data_cycle['spot']==spot)&(atl06_data_cycle['rgt']==rgt)]['geometry'].y.values
            lon = atl06_data_cycle[(atl06_data_cycle['spot']==spot)&(atl06_data_cycle['rgt']==rgt)]['geometry'].x.values
            tform = Transformer.from_crs('EPSG:4326','EPSG:3031',always_xy=True)
            [E,N] = tform.transform(lon,lat)
            if len(h)>100:
                d = atl06_data_cycle[(atl06_data_cycle['spot']==spot)&(atl06_data_cycle['rgt']==rgt)]['distance'].values
                dictionary = {"cycle":cycle,"rgt":rgt,"spot":spot,"d":d,"h":h,"E":E,"N":N,"E_out":[],"N_out":[],"R":[]}
                dictionary = pd.DataFrame([dictionary])
                df = pd.concat([df,dictionary],ignore_index=True)
    return(df)

In [ ]:
# set parameters for roughness calculation
sz = 1000
slide = 100
klims = [1/1000,1/90]
shelf = 'pig'
num_cycles = 16

#make empty dataframe
meta_df = pd.DataFrame()

#iterate through each individual cycle
for i in range(1,num_cycles):
    
    #subset to just cycle n and make formated dataframe
    atl06_pig_subset = atl06_pig[(atl06_pig['cycle']==i)]
    meta_df_i = make_dataframe(atl06_pig_subset,i)
    
    #do roughness calculation for all lines in one cycle, then append to main dataframe
    for j in range(np.shape(meta_df_i)[0]):
        R,R_detrend,E_out,N_out = sliding_roughness(meta_df_i[j:j+1],sz,slide,100,10,5,klims)
        meta_df_i.at[j,'R'], meta_df_i.at[j,'E_out'], meta_df_i.at[j,'N_out'], = R_detrend, E_out, N_out
    
    #make dataframe for each cycle
    with open('/home/acdealy/notebooks/ice-shelf-roughness/outputs/'+shelf+'detrend'+'_df_cycle_'+str(i), 'wb') as handle:
        pickle.dump(meta_df_i, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#make empty dataframe
meta_df = pd.DataFrame()

#iterate through each cycle and concatenate each datafram
for i in range(1,16):
    
    #read dataframe in format for plotting
    meta_df_i = pd.read_pickle('/home/acdealy/notebooks/ice-shelf-roughness/outputs/'+shelf+'detrend'+'_df_cycle_'+str(i))
    meta_df = pd.concat([meta_df,meta_df_i])

#save dataframe with all cycles
with open('/home/acdealy/notebooks/ice-shelf-roughness/outputs/'+shelf+'detrend'+'_df_cycle_all', 'wb') as handle:
    pickle.dump(meta_df, handle, protocol=pickle.HIGHEST_PROTOCOL)